# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py F1AP.docx 'keyword'" for F1AP and "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py NGAP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: NAS messages with mismatched UE identities accepted without validation, enabling potential security bypass.
Repro Steps:
Establish UE context with AMF-UE-NGAP-ID=100, RAN-UE-NGAP-ID=200, and UE identity in NAS layer as SUPI-A.
Send UPLINK NAS TRANSPORT message with AMF-UE-NGAP-ID=100, RAN-UE-NGAP-ID=200, but NAS-PDU containing different UE identity SUPI-B.
OAI NG-RAN accepts the message

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "NAS messages with mismatched UE identities (SUPI-A vs SUPI-B) are accepted by NG-RAN without validation during UPLINK NAS TRANSPORT.",
    "interface_guess": ["NGAP", "NAS"],
    "procedure_guess": ["UPLINK NAS TRANSPORT"],
    "components_involved": ["gNB-CU", "AMF"],
    "key_ids": {
      "transaction_id": null,
      "cu_ue_f1ap_id": null,
      "du_ue_f1ap_id": null,
      "du_id": null,
      "rnti": null,
      "pci": null,
      "served_cell_list": [],
      "amf_ue_ngap_id": "100",
      "ran_ue_ngap_id": "200"
    },
    "signals_or_timers": []
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": ["UPLINK NAS TRANSPORT", "transparent"],
      "title": ["TS 38.413 8.6.2 Uplink NAS Transport"],
      "expected_behaviour": "The NG-RAN node shall transfer the NAS-PDU IE transparently to the AMF."
    },
    {
      "Keywords": ["NAS-PDU", "Interpretation"],
      "title": ["TS 38.413 9.3.4.1 NAS-PDU"],
      "expected_behaviour": "The NAS-PDU IE contains an AMF-UE message that is transferred without interpretation in the NG-RAN node."
    },
    {
      "Keywords": ["ULInformationTransfer"],
      "title": ["TS 38.331 5.7.2 UL information transfer"],
      "expected_behaviour": "Transfers NAS dedicated information from UE to Network."
    }
  ],
  "expected_behaviour": "The gNB should transparently forward the NAS-PDU received in RRC ULInformationTransfer to the AMF using NGAP UPLINK NAS TRANSPORT, without inspecting the NAS-PDU contents."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "function_name": "rrc_gNB_send_NGAP_UPLINK_NAS",
      "reason": "Handles the construction of NGAP UPLINK NAS TRANSPORT message from RRC."
    },
    {
      "path": "openair2/RRC/NR/rrc_gNB.c",
      "function_name": "rrc_forward_ue_nas_message",
      "reason": "Handles DL forwarding, relevant for context of transparent forwarding implementation."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "code",
      "source": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "location": "725:752",
      "text": "void rrc_gNB_send_NGAP_UPLINK_NAS(...) { ... MessageDef *msg_p = itti_alloc_new_message(TASK_RRC_GNB, rrc->module_id, NGAP_UPLINK_NAS); ... NGAP_UPLINK_NAS(msg_p).nas_pdu.buf = buf; ... itti_send_msg_to_task(TASK_NGAP, rrc->module_id, msg_p); }"
    },
    {
      "kind": "spec",
      "source": "TS 38.413",
      "location": "8.6.2",
      "text": "The NAS-PDU IE contains an AMF–UE message that is transferred without interpretation in the NG-RAN node."
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    { "from": "UE", "to": "gNB", "message": "RRC: ULInformationTransfer(NAS-PDU)" },
    { "from": "gNB", "to": "AMF", "message": "NGAP: UPLINK NAS TRANSPORT(NAS-PDU)" },
    { "from": "AMF", "to": "gNB", "message": "NGAP: DOWNLINK NAS TRANSPORT (Optional Reply)" }
  ],
  "state_machine": {
    "states": ["RRC_CONNECTED", "RRC_MSG_RECEIVED", "NGAP_MSG_CONSTRUCTED", "WAIT_NGAP_TX", "NAS_RELAYED"],
    "transitions": [
      { "from": "RRC_CONNECTED", "to": "RRC_MSG_RECEIVED", "on": "ULInformationTransfer", "guard": "Valid RRC" },
      { "from": "RRC_MSG_RECEIVED", "to": "NGAP_MSG_CONSTRUCTED", "on": "Extract NAS-PDU", "guard": "NAS-PDU Present" },
      { "from": "NGAP_MSG_CONSTRUCTED", "to": "WAIT_NGAP_TX", "on": "ITTI Send", "guard": "None" },
      { "from": "WAIT_NGAP_TX", "to": "NAS_RELAYED", "on": "Msg Sent to AMF", "guard": "None" }
    ]
  },
  "assumptions": ["gNB treats NAS PDU as opaque blob per TS 38.413 8.6.2."],
  "questions": ["Is the reporter expecting AMF behavior but testing via gNB?"],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "NAS Transparency",
      "result": "PASS",
      "evidence": ["TS 38.413 8.6.2", "rrc_gNB_send_NGAP_UPLINK_NAS implementation"]
    },
    {
      "name": "Identity Validation (Reporter Expectation)",
      "result": "FAIL",
      "evidence": ["Code does not validate NAS content, which is correct per spec"]
    }
  ],
  "preliminary_verdict": "ambiguous",
  "rationale": "Code is spec-compliant (transparent forwarding). The reported issue (lack of validation) is actually the correct behavior for gNB. Validation belongs in AMF."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": ["TS 38.413 8.6.2"],
    "code_citations": ["openair2/RRC/NR/rrc_gNB_NGAP.c:725"],
    "expected_vs_observed": "Expected by Spec: Transparent forwarding of NAS-PDU. Observed: Transparent forwarding. Expected by Reporter: Validation in RAN. The observed behavior matches the spec, contradicting the reporter's expectation."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "ambiguous",
    "rationale": "OAI NG-RAN correctly implements the transparent forwarding of NAS messages as required by TS 38.413. The security validation of NAS-PDU identities must be performed by the AMF. This report suggests the bug is either invalid for RAN or should be transferred to Core Network.",
    "citations": {
      "spec": ["TS 38.413 8.6.2"],
      "code": ["openair2/RRC/NR/rrc_gNB_NGAP.c"]
    },
    "suggested_repro": ["Check AMF logs for NAS security validation failure."],
    "risks": ["Potential security bypass if AMF also fails to validate."],
    "next_steps": ["Investigate OAI-CN (AMF) handling of mismatched NAS identities."]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.